# Hypothesis - public transport serves people of relative wealth

Hey all, back once again with another banger of a hypothesis. Making a real effort to document the entire process including my thought process, so that every step and every assumption is bare for all to see and improve upon.

I want to use the data we have available to us to provide a response to my stated hypothesis of "public transport serves people of relative wealth." This is possibly the most simple of the hypothesis we've come up with so far (as of Tuesday 8th, 8:30pm), so it's possibly not the best one to use for the full example write-up - but hopefully reading the process here will lay bare the process that went into developing the other Jupyter notebooks I've already posted.

To prove or disprove this hypothesis, I want to prove/disprove that there is a relationship between the availability of public transport services (as represented by the number of services that have local stops from dataset 1A) grouped within each SA2 area (by joining 1A with 1F) to see which of three outcomes is true:
- areas of relative socioeconomic advantage (Steff -> Kate's areas) have more public transport services in their areas (hypothesis is at least _mostly_ true)
- areas of relative socioeconomic disadvantage (Eric -> Charlie's areas) have more public transport services in their areas (counter-hypothesis is at least _mostly_ true)
- there is no discernable pattern between socioeconomic status and the availability of public transport services (hypothesis is unproven)

It's worth also noting that I am not attempting to explain the relationship either proven or disproven by this hypothesis - we don't have access to the data to do that in any sort of meaningful way. The best we can do at this point is to prove that a relationship exists, not that it is causal/correlated/corraborated/and so on.

The other thing I am looking at including in this hypothesis (at least until I get into it to see if it's possible/worth the effort to do) is to further examine whether this hypothesis is more or less relevant for each of the methods of travel present in the data - bus, train, tram and ferry. My assumption going in is that there may not be enough tram or ferry stops in areas of differing socio-economic status to draw any meaninful conclusions (given that trams ONLY run on the Gold Coast and ferry's ONLY run in those areas of the CBD by the river). Will still test to see if anything meaningful comes out here.
Of greater interest is comparing buses and trains - might turn up nothing, might turn up something new and interesting we can further explore.

Finally, unlike the other hypothesis tested so far, I cannot think of a meaningful way to extend this analysis by considering each of our SA2 areas separately - I'm already doing this from the start (either explicitly by sorting on Team_Member or implicitly by using raw SAD index scores), so I can probably leave this off from my normal workflow (or, at least, that's my current thinking about it. This may change as I get further in).

My current workflow involves having both this Jupyter notebook and R Studio open in parallel. I perform every step in R Studio first and make sure it does what I want it to do (and correct anything that needs correcting), then paste the code in here to ensure Jupityer gives an identical result.

First, I add the libraries I intend to use (and at this point, they are just the libraries I have been using in playing with the data). I might not use one of these but doing this consistently every time means I can have some faith that it will produce consistent results (with the exception of the time I loaded the libraries in a different order and broke everything!)

In [1]:
library(readr)
library(plyr)
library(dplyr)
# Note from Dave to Dave - ALWAYS load plyr before dplyr or group_by doesn't work properly
# Note - have updated to use combined_quantity from the 1C dataset to give more meaningful results

Warning message:
"package 'readr' was built under R version 3.4.4"Warning message:
"package 'plyr' was built under R version 3.4.4"
Attaching package: 'dplyr'

The following objects are masked from 'package:plyr':

    arrange, count, desc, failwith, id, mutate, rename, summarise,
    summarize

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union



Firstly, I want to bring in dataset 1A. I have been playing with it a little to become familiar with it this afternoon, but now I want to start from scratch and apply transformations on it here so everyone can reproduce it exactly.
For this step, I am using the output.txt that exists in the 1A Final Data folder on Google Drive (as of 8/05) - if another version is added later, I'll try to update my code here. If you try running it and it DOESN'T work, let me know because I've probably just forgotten =p

Also, heads up - this next codeblock might take a while to run, 1A is a bit of a monster dataset.

In [2]:
output <- read_csv("Processed Data/1A/output.txt")
head(output)

Parsed with column specification:
cols(
  trip_id = col_character(),
  arrival_time = col_time(format = ""),
  departure_time = col_time(format = ""),
  stop_id = col_integer(),
  stop_sequence = col_integer(),
  pickup_type = col_integer(),
  drop_off_type = col_integer(),
  is_weekend = col_character(),
  is_duplicated = col_character(),
  arr_hr = col_integer(),
  arrival_time_24 = col_datetime(format = ""),
  arrival_time_bin = col_integer()
)
Warning message in rbind(names(probs), probs_f):
"number of columns of result is not a multiple of vector length (arg 1)"Warning message:
"52486 parsing failures.
row # A tibble: 5 x 5 col     row col            expected   actual   file                           expected   <int> <chr>          <chr>      <chr>    <chr>                          actual 1   542 arrival_time   valid date 24:02:00 'Processed Data/1A/output.txt' file 2   542 departure_time valid date 24:02:00 'Processed Data/1A/output.txt' row 3   543 arrival_time   valid date 24:0

trip_id,arrival_time,departure_time,stop_id,stop_sequence,pickup_type,drop_off_type,is_weekend,is_duplicated,arr_hr,arrival_time_24,arrival_time_bin
9567471-BBL 17_18-399-Weekday-01,06:30:00,06:30:00,12096,1,0,0,False,False,6,2018-05-07 06:30:00,1
9567471-BBL 17_18-399-Weekday-01,06:39:00,06:39:00,313180,2,0,0,False,False,6,2018-05-07 06:39:00,1
9567471-BBL 17_18-399-Weekday-01,06:40:00,06:40:00,313177,3,0,0,False,False,6,2018-05-07 06:40:00,1
9567471-BBL 17_18-399-Weekday-01,06:45:00,06:45:00,316811,4,0,0,False,False,6,2018-05-07 06:45:00,1
9567471-BBL 17_18-399-Weekday-01,06:50:00,06:50:00,316812,5,0,0,False,False,6,2018-05-07 06:50:00,1
9567471-BBL 17_18-399-Weekday-01,06:51:00,06:51:00,316814,6,0,0,False,False,6,2018-05-07 06:51:00,1


Now that it's imported and we can see it, I can see there's a number of columns I simply don't need to do the analysis I want to do. In fact, I am thinking about it now, and the dataset I need to be effective in my analysis on this hypothesis should have:
- stop_id (can't do anything without good ol' stop_id)
- arrival_time_bin (1-5, corresponding to the times from the TransLink Origin-Destination dataset (1C_trip_freq). I'll paste these below this list for reference)
- a calculated column which has the number of services per stop_id, per arrival_time_bin (all I'm really after is the number of services that stop at each stop_id).

I _technically_ don't even need arrival_time_bin, but I'm going to carry it forward so I can use the resulting dataset for other things later (specifically, to use instead of number of stops in my "Hypothesis - public transport is designed to serve people who use it to commute (for work, school, etc)" file.

- bin 1 - Weekday (12:00am-8:29:59am)
- bin 2 - Weekday (8:30am-2:59:59pm)
- bin 3 - Weekday (3:00pm-6:59:59pm)
- bin 4 - Weekday (7:00pm-11:59:59pm)
- bin 5 - Weekend

As a result, I know I can already start dropping columns that I'm just not going to use, so let's do that now.

In [3]:
output$trip_id <- NULL
output$arrival_time <- NULL
output$departure_time <- NULL
output$stop_sequence <- NULL
output$pickup_type <- NULL
output$drop_off_type <- NULL
output$is_duplicated <- NULL
output$arr_hr <- NULL
output$arrival_time_24 <- NULL
head(output)

stop_id,is_weekend,arrival_time_bin
12096,False,1
313180,False,1
313177,False,1
316811,False,1
316812,False,1
316814,False,1


As you can see, we're now left with a much smaller and easier to work with dataset that contains only:
- stop_id
- is_weekend
- arrival_time_bin

You might wonder why I've thrown out is_duplicated so early - now that we aren't looking to calculate the difference in time between each unique departure, duplicates aren't duplicates. By leaving in duplicates, I have the raw count of the number of services that used each stop, divided into the five time bins, for one standard week.

Side fact - this dataset also contains _abnormal_ trips, such as public holiday timetables. We spent some time looking at whether this were reasonable to remove, but found it impractical to do without access to additional data that isn't openly available. We were also influenced by just _how often_ TransLink put on abnormal services - https://translink.com.au/plan-your-journey/event-transport

I can count 9 in the next fortnight. So we can either clean these data through reference to another dataset that we don't have access to, or we can recognise that there are data quality issues that prevent these counts from being 100% accurate in some cases - ultimately we were comfortable with the compromise of this issue for the purposes of our analysis, which is all based on relative comparsion and not tangible figures. If we were using this in anger, or in a PhD submission, we would have come up with another answer and either solved this problem or found alternative data.

Next, to get this data to where I need it to be, I need to correct _another_ data quality issue - all weekend trips are designated by "True" in the is_weekend column but are NOT given a bin of 5. Let me demonstrate first -

In [4]:
summary(output)

    stop_id        is_weekend        arrival_time_bin
 Min.   :     1   Length:2256935     Min.   :1.000   
 1st Qu.:  4436   Class :character   1st Qu.:2.000   
 Median :300005   Mode  :character   Median :2.000   
 Mean   :186257                      Mean   :2.388   
 3rd Qu.:310054                      3rd Qu.:3.000   
 Max.   :600856                      Max.   :4.000   

As you can see, arrival_time_bin has a max of 4 - weekends aren't being picked up correctly. Let's fix that first.

In [5]:
output$arrival_time_bin[output$is_weekend == "True"] <- 5
summary(output)

    stop_id        is_weekend        arrival_time_bin
 Min.   :     1   Length:2256935     Min.   :1.000   
 1st Qu.:  4436   Class :character   1st Qu.:2.000   
 Median :300005   Mode  :character   Median :4.000   
 Mean   :186257                      Mean   :3.513   
 3rd Qu.:310054                      3rd Qu.:5.000   
 Max.   :600856                      Max.   :5.000   

I ran the summary(output) to make sure our changes took - you can now see the Max for arrival_time_bin is 5. As a result, is_weekend has served it's purpose and I can dump it overboard as well.

In [6]:
output$is_weekend <- NULL
head(output)

stop_id,arrival_time_bin
12096,1
313180,1
313177,1
316811,1
316812,1
316814,1


As you can see, I'm now working with a very narrow but deep dataset (for all these manipulations, it still has over 2 million rows). Last step I want to take is to aggregate these unit-record data to their summarised form as described above, which has a count of the number of weekly services per stop_id per arrival_time_bin. I'll also finally give our dataset a more meaningful name (output? not super descriptive?)

In [7]:
pt_services <- output %>% group_by (stop_id, arrival_time_bin) %>% count(stop_id)
head(pt_services)
# write.csv(pt_services,"pt_services.csv", row.names = FALSE)

stop_id,arrival_time_bin,n
1,1,60
1,2,150
1,3,80
1,4,34
1,5,313
2,1,56


And there we have it, my final dataset to combine with 1F for this piece of analysis. I've written it out to a CSV file that I'll upload to the Dave Output folder on Google Drive now, so if you'd like to get hands on it but don't want to do it yourself, that's where you can find it - or just uncomment that line and run that code block to get your own).

https://drive.google.com/open?id=1uEME6MIomJFddB9zltw2hECfz7rtmT37

It's also reduced our dataset from 2 million rows to around 55k - a little easier to work with again.

So, let's have a quick look at this new dataset's summary stats to see if we learn anything interesting.

In [8]:
summary(pt_services)

    stop_id       arrival_time_bin       n          
 Min.   :     1   Min.   :1.000    Min.   :   1.00  
 1st Qu.:  4733   1st Qu.:2.000    1st Qu.:   7.00  
 Median :300379   Median :3.000    Median :  16.00  
 Mean   :176388   Mean   :2.897    Mean   :  41.14  
 3rd Qu.:311532   3rd Qu.:4.000    3rd Qu.:  38.00  
 Max.   :600856   Max.   :5.000    Max.   :6809.00  

There is (at least one) stop that only had a single service on the timetable, (at least) one stop that had 6,809 stops a week. Busy stop! We can also see the median of 16 and the mean of 41.14 - having such a large variation suggests that this data is left leaning on the standard deviation curve.  Cool (but not super helpful by itself - we'll remember this for later).

Next, let's take what we've learned here and aggregate them up to the SA2 area level we are using for our analysis. To do this, it's time to bring in 1F. To do THAT, I'm going to cheat a little and bring in my 1F file (which is available under Data - Processed Data - 1F - SA2_stops_by_mode.csv) I should do the right thing and bring in the 1F file from the Final Data column, but I'm only going to exclude all of the rows that are different from my file anyway and I've shown my approach to doing that above, so I'm going to streamline a little.

Because I already know what I want from 1F, I'm going to import it to skip a bunch of the columns from my file upfront.

In [9]:
SA2_stops_by_mode <- read_csv("Processed Data/1F/SA2_stops_by_mode.csv", col_types = cols(AREASQKM16 = col_skip(), Pop = col_skip(), stop_lat = col_skip(), stop_lon = col_skip(), stop_name = col_skip(), stop_url = col_skip()))
head(SA2_stops_by_mode)

stop_id,SA2Code,SA2Name,Score,Team_Member,is_bus_stop,is_ferry_stop,is_tram_stop,is_train_station
1,305011105,Brisbane City,1083,Steff,1,0,0,0
2,305011105,Brisbane City,1083,Steff,1,0,0,0
3,305011105,Brisbane City,1083,Steff,1,0,0,0
4,305011111,Spring Hill,1028,Dave,1,0,0,0
5,305011105,Brisbane City,1083,Steff,1,0,0,0
6,305011105,Brisbane City,1083,Steff,1,0,0,0


You can see the columns I've selected to keep, and here's why:
- stop_id (can't do anything without good ol' stop_id) (also I need it to do the matching with our first dataset)
- SA2Code (need this as the datapoint to aggregate everything to for this hypothesis)
- SA2Name (just for readability later on)
- Score (will use this first to see if I can find a pattern, then might revert back to Team_Member instead based on what I find)
- is_bus_stop (in case I want to do my extension bit at the end)
- is_ferry_stop (in case I want to do my extension bit at the end)
- is_tram_stop (in case I want to do my extension bit at the end)
- is_train_station (in case I want to do my extension bit at the end)

For the record, it's personal preference why I prefer working with the smallest viable dataset when I'm doing my analysis - less likely to confuse myself, less likely to get distracted and try to solve a different question half-baked, more computationally-effecient to work with smaller files. If you wanted to import the whole dataset (or even the monster 1F dataset from the Final folder) no worries at all, the next few code blocks will get us where we need to anyway (just slower, I guess).

Next, I want to join our two working datasets together. I specifically use inner joins (which only joins on records that have a matching stop_id in both tables, because there's no value for this specific hypothesis in finding any stops that don't have a record in pt_services (although that would be interesting - why are they stops if they aren't used for anything? But not the point I'm working on here. Might be a good exploratory piece if someone is looking for inspiration to sink their teeth into!

Also, I am using join, but I also use a fair amount of merge() to achieve similar things - I generally use merge only if I'm joining two of our "read-only" datasets so I can get all the warts.

In [10]:
pt_services_by_SA2 <- join(SA2_stops_by_mode, pt_services, by="stop_id", type="inner")
head(pt_services_by_SA2)

stop_id,SA2Code,SA2Name,Score,Team_Member,is_bus_stop,is_ferry_stop,is_tram_stop,is_train_station,arrival_time_bin,n
1,305011105,Brisbane City,1083,Steff,1,0,0,0,1,60
1,305011105,Brisbane City,1083,Steff,1,0,0,0,2,150
1,305011105,Brisbane City,1083,Steff,1,0,0,0,3,80
1,305011105,Brisbane City,1083,Steff,1,0,0,0,4,34
1,305011105,Brisbane City,1083,Steff,1,0,0,0,5,313
2,305011105,Brisbane City,1083,Steff,1,0,0,0,1,56


Merged table is great, now to aggregate to the SA2 level.

In [13]:
pt_services_by_SA2 <- pt_services_by_SA2 %>% group_by (SA2Code, SA2Name, Score, Team_Member, arrival_time_bin) %>% summarise(n = sum(n))
head(pt_services_by_SA2)

SA2Code,SA2Name,Score,Team_Member,arrival_time_bin,n
112031254,Tweed Heads,933,Eric,1,620
112031254,Tweed Heads,933,Eric,2,1060
112031254,Tweed Heads,933,Eric,3,630
112031254,Tweed Heads,933,Eric,4,365
112031254,Tweed Heads,933,Eric,5,1792
301011001,Alexandra Hills,987,Charlie,1,358


And just like that, every SA2 code and the number of trips per time bin. Lets check some smexy summary stats on that count...

In [12]:
summary(pt_services_by_SA2$n)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
      4     335     779    1549    1723   46713 

The thing that immediately jumps out to me is the difference between the 3rd quintile (1,089) and the max (42,622) - we have some bigggg outliers in area/time_bin combinations. Let's run a quick thought experiement, aggregate all of the bins together and see if that trend is a one off or strengthened (note - this isn't vital to answering this hypothesis, but I'm interested to see what that looks like and whether it might be more fertile ground for us to explore further).

In [14]:
pt_services_by_SA2_v2 <- pt_services_by_SA2 %>% group_by (SA2Code, SA2Name, Score, Team_Member) %>% summarise(n = sum(n))
summary(pt_services_by_SA2_v2$n)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
      8    2582    4670    7599    8631  106254 

You can see that the extra aggregation is as simple as removing "arrival_time_bin" from the group_by area - it just doesn't group by that. Pretty easy to do =)
So what did that tell us? 3rd quantile score increased 5.5x, Max only increased by 2.1x - now that _is_ interesting! Something for another time though...

_If anyone wants to pick this up and run with it, please do - I would love to know what's going on there!_

And this is the part where there's a few things that I'd normally do. 
- First is to export this dataset to import to Tableau, with a view of plotting the SAD index scores vs. the count of services per area. If it looks like there's a clear pattern, great - write it up and assess the original hypothesis
- If the visual EDA doesn't show a clear pattern, but has the hints there is a pattern to be found, I'd come back to R Studio/Jupityer notebooks, filter the data by Team_Member and start doing comparison there. The extra level of aggregation might reveal a clearer pattern that is otherwise obscured by outliers at the SA2 level. If that turns up a pattern - great, write it up as a weaker relationship (but still a relationship) and assess the original hypotheses.
- If there's no pattern at that point - see what it does reveal to see if there's anything else I'd like to explore futher - possibly there is something that hints at _another_ hypothesis we can test. 
- If not that either - write it up as hypothesis is unproven. Not a bad outcome and not a failure - we now know something else about the world as revealed by this data (just not something that we can action in any meaningful way).

With that captured, let's get doin' in Tableau. Exporting the v2 dataset and will post the resulting plot in a sec...

In [15]:
# write.csv(pt_services_by_SA2_v2,"pt_services_by_SA2_v2.csv", row.names = FALSE)

Exported dataset is in Dave Output folder if you don't want to create it yourself - https://drive.google.com/open?id=1-imrefuYw2jQFBBbpZJPK33iMjr07voW

Ok - have done some quick plotting in Tableau (probably would benefit from learning how to plot directly in R, but that's something for another day).

Interestingly, there appears to be some validity to this hypothesis - here's the export - https://github.com/TheDataStarter/Lets-Go-card-/blob/master/viz/SAD%20index%20score%20vs%20number%20of%20services.png
(It's SAD Index Score vs Number of Services.png in Project - Analysis - Dave - Dave Vis.

To make sure the trend line wasn't being overly influenced by the outliers (which are across the board but the most impactful are at the top end of the scores), I ran the same plot with any SA2 area with more than 20k weekly services excluded. Even with this, the same trend line can be seen - https://github.com/TheDataStarter/Lets-Go-card-/blob/master/viz/SAD%20index%20score%20vs%20number%20of%20services%20(2).png

(same as file above, but with a (2) at the end of the filename.

In both cases, there is a clear trend that shows that the number of services in an area increases with the SAD index score for the area. At this point - hypothesis _mostly_ proven! 

Now, let's do the R piece anyway, which as described above involves filtering on our SA2 areas rather than SAD index score and comparing summary stats.

In [19]:
# Note - to use this code block, only uncomment one of the indiv_data lines. Trying to use more than one will overwrite with 
# the last one and not give the desired result.

# indiv_data <- subset(pt_services_by_SA2_v2, Team_Member == "Eric") # only consider Eric's SA2 areas
# indiv_data <- subset(pt_services_by_SA2_v2, Team_Member == "Charlie") # only consider Charlie's SA2 areas
# indiv_data <- subset(pt_services_by_SA2_v2, Team_Member == "Dave") # only consider my SA2 areas
# indiv_data <- subset(pt_services_by_SA2_v2, Team_Member == "Kate") # only consider Kate's SA2 areas
# indiv_data <- subset(pt_services_by_SA2_v2, Team_Member == "Steff") # only consider Steff's SA2 areas

# Once you've selected A name, use the below to either summarise or export. (Or, y'know, whatever you want to do with it 
# from here))

# write.csv(indiv_data,"indiv_data.csv", row.names = FALSE)

 summary(pt_services_by_SA2_v2$n)
# summary(indiv_data$n)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
      8    2582    4670    7599    8631  106254 

Textual summary for anyone reading the non-interactive version of this notebook:
- all SA2s - have between 8 and 106,254 services per area per week, median of 4,670, mean of 7,599
- Eric's SA2s - have between 24 and  45,499 services per area per week, median of 3,911, mean of 5,243 (fewer services than average)
- Charlie's SA2s - have between 8 and 50,430 services per area per week, median of 3,486, mean of 6,805 (fewer services than average)
- my SA2s - have between 48 and 41,185 services per area per week, median of 4,990, mean of 7,872 (more than average)
- Kate's SA2s - have between 364 and 32,435 services per area per week, median of 5,000, mean of 7,396 (more than average)
- Steff's SA2s - have between 65 and 106,254 services per area per week, median of 6,734, mean of 10,676 (more than average but clearly benefiting from the effect of outliers

### Aggregating to our areas actually serves to obscure the relationship a little, which is interesting in itself. As a result, I think we should use the Tableau visual EDA as evidence of our hypothesis in this case and call this one a meaningful hypothesis.

# Peace